# Custom Symmetry Group Example

In [ ]:
import symd
import matplotlib.pyplot as plt
import numpy as np

### Make custom group

Rotational group with reflection boundary conditions. Asymmetric unit is just a guess - not actually correct, so we may have overlapping particles and need to re-run the simulation.

In [ ]:
rot = 7
genpos = []
for ri in range(rot):
    c = np.round(np.cos(ri * np.pi * 2 / rot), 4)
    s = np.round(np.sin(ri * np.pi * 2 / rot), 4)
    if ri == 1:
        # asymm = f'0≤x≤{c};0≤y≤{s}'
        asymm = f"1/2≤x≤1.0;1/2≤y≤1.0"
    genpos.append(
        f"{c}x {-s:+}y {-0.5 * c + 0.5 * s + 0.5:+},{s}x {c:+}y {-0.5 * s - 0.5 * c + 0.5:+}"
    )
specpos = [symd.Group("Oblique", ["1/2, 1/2"], specpos=[])]

my_group = {
    "lattice": "Oblique",
    "genpos": genpos,
    "asymm_unit": asymm,
    "specpos": specpos,
}

my_group = symd.Group("Oblique", genpos, specpos, "quasi", asymm)

### Run

Run the simulation

In [ ]:
def run_sim(
    n, number_density, group, images, w=None, retries=50, pos_frames=0, steps=10**6
):
    for i in range(retries):
        print("Trying on ", i)
        try:
            np.random.seed(i)
            cell = symd.groups.get_cell(number_density, group, 2, n, w)
            md = symd.Symd(
                nparticles=n,
                cell=cell,
                ndims=2,
                images=images,
                force="lj",
                wyckoffs=w,
                group=group,
                steps=steps,
                exeDir="quasi",
                start_temperature=0.5,
                temperature=0.1,
                pressure=0.25,
            )
            md.remove_overlap()
            md.runParams["box_update_period"] = 10
            md.runParams["langevin_gamma"] = 0.5
            md.log_positions(frames=pos_frames)
            try:
                md.run()
            except RuntimeError as e:
                d = md.number_density()
                if d < 0.6:
                    print("Not dense enough, retrying", d)
                    continue

            # Basically E-min
            md.runParams["start_temperature"] = 0.1
            md.runParams["temperature"] = 1e-2
            md.runParams["langevin_gamma"] = 0.5
            md.runParams["Pressure"] = None
            md.runParams["box_update_period"] = 0
            md.runParams["steps"] = steps // 10
            if pos_frames > 0:
                md.log_positions(filename="equil.xyz", frames=pos_frames // 10)
            try:
                md.run()
            except RuntimeError as e:
                continue
            config = md.positions[-1]
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

In [ ]:
md = run_sim(256, 0.1, my_group, [0, 0], pos_frames=1000, w=[1])

In [ ]:
plt.plot(md.positions[-1, :, 0], md.positions[-1, :, 1], ".")